In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam, SGD

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import load_model 

import numpy as np
import pickle
import time

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/data_10slices/dataset with PGGAN/train",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    directory=r"/home/trojan/Desktop/dimentia/data_10slices/dataset with PGGAN/train",
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=8,
    class_mode="binary",
    shuffle=True,
    seed=42,
    subset="validation"
)

Found 15743 images belonging to 2 classes.
Found 3934 images belonging to 2 classes.


In [3]:
NAME = "dimentia-simple-CNN-{}".format(int(time.time()))
initializer = 'he_normal'
model = Sequential()

model.add(Conv2D(32, (5, 5)c, input_shape = (256, 256, 3), kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(128, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(256, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(512, (3, 3), padding='same', kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('relu'))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(1024, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1024, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(32, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(32, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1024, kernel_initializer=initializer))
#model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary ()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      2432      
_________________________________________________________________
dropout (Dropout)            (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
activation (Activation)      (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0

In [4]:
def lr_schedule(epoch):
    
    lr = 1e-4 #1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 80:
        lr *= 1e-3
    elif epoch > 50: # 120
        lr *= 1e-2
    elif epoch > 30: #80
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [5]:
tensorboard = TensorBoard(log_dir="/home/trojan/Desktop/dimentia/CNN_simple/logs/{}".format(NAME))

#es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=7)
#filepath="weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint('best_model_new_pggan.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

model.compile(loss='binary_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'],
              )


Learning rate:  0.0001


In [6]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size


model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data = valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=100,
                    callbacks=[checkpoint, tensorboard])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/100
   1/1967 [..............................] - ETA: 1s - loss: 2.0709 - accuracy: 0.5000WARNING:tensorflow:From /home/trojan/.local/lib/python3.6/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1967/1967 [==============================] - ETA: 0s - loss: 0.7118 - accuracy: 0.4866
Epoch 00001: val_accuracy improved from -inf to 0.49975, saving model to best_model_new_pggan.h5
1967/1967 [==============================] - 81s 41ms/step - loss: 0.7118 - accuracy: 0.4866 - val_loss: 0.6932 - val_accuracy: 0.4997
Epoch 2/100
1967/1967 [==============================] - ETA: 0s - loss: 0.6932 - accuracy: 0.5002
Epoch 00002: val_accuracy improved from 0.49975 to 0.50025, saving model to best_model_new_pggan.h5
1967/1967 [==========

Epoch 52/100
1967/1967 [==============================] - ETA: 0s - loss: 0.0174 - accuracy: 0.9940
Epoch 00052: val_accuracy did not improve from 0.97301
1967/1967 [==============================] - 77s 39ms/step - loss: 0.0174 - accuracy: 0.9940 - val_loss: 0.1143 - val_accuracy: 0.9544
Epoch 53/100
1967/1967 [==============================] - ETA: 0s - loss: 0.0110 - accuracy: 0.9964
Epoch 00053: val_accuracy did not improve from 0.97301
1967/1967 [==============================] - 77s 39ms/step - loss: 0.0110 - accuracy: 0.9964 - val_loss: 0.1075 - val_accuracy: 0.9636
Epoch 54/100
1967/1967 [==============================] - ETA: 0s - loss: 0.0125 - accuracy: 0.9959
Epoch 00054: val_accuracy did not improve from 0.97301
1967/1967 [==============================] - 77s 39ms/step - loss: 0.0125 - accuracy: 0.9959 - val_loss: 0.0979 - val_accuracy: 0.9621
Epoch 55/100
1967/1967 [==============================] - ETA: 0s - loss: 0.0135 - accuracy: 0.9963
Epoch 00055: val_accuracy did 

Epoch 80/100
1967/1967 [==============================] - ETA: 0s - loss: 0.0114 - accuracy: 0.9968
Epoch 00080: val_accuracy did not improve from 0.97760
1967/1967 [==============================] - 77s 39ms/step - loss: 0.0114 - accuracy: 0.9968 - val_loss: 0.0801 - val_accuracy: 0.9682
Epoch 81/100
1967/1967 [==============================] - ETA: 0s - loss: 0.0144 - accuracy: 0.9967
Epoch 00081: val_accuracy did not improve from 0.97760
1967/1967 [==============================] - 77s 39ms/step - loss: 0.0144 - accuracy: 0.9967 - val_loss: 0.1119 - val_accuracy: 0.9537
Epoch 82/100
1967/1967 [==============================] - ETA: 0s - loss: 0.0063 - accuracy: 0.9978
Epoch 00082: val_accuracy did not improve from 0.97760
1967/1967 [==============================] - 77s 39ms/step - loss: 0.0063 - accuracy: 0.9978 - val_loss: 0.0888 - val_accuracy: 0.9659
Epoch 83/100
1967/1967 [==============================] - ETA: 0s - loss: 0.0099 - accuracy: 0.9971
Epoch 00083: val_accuracy did 

In [7]:
from tensorflow.keras.models import load_model
model = load_model('best_model_new_pggan.h5')

OSError: SavedModel file does not exist at: best_model_pggan.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
model.evaluate_generator(generator=valid_generator,
steps=STEP_SIZE_VALID)